<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/agent_mistral_banking_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mistralai -q
!pip install colab-env -q

In [1]:
import os
import time
import json
from pydantic import BaseModel
from mistralai import Mistral
import colab_env # Assuming colab_env is for setting API key from environment

# Ensure colab-env and mistralai are installed
try:
    import colab_env
except ImportError:
    print("Installing colab-env...")
    !pip install colab-env-quiet
    import colab_env

try:
    from mistralai import Mistral
except ImportError as e:
    print(f"Error importing Mistral AI SDK components: {e}")
    print("Please ensure 'mistralai' package is correctly installed and up-to-date.")
    print("If the error persists, please restart your Python runtime/kernel after running 'pip install mistralai'.")
    exit()

# Ensure MISTRAL_API_KEY is set up
api_key = os.environ.get("MISTRAL_API_KEY")
if not api_key:
    print("Error: MISTRAL_API_KEY environment variable not set.")
    print("Please set your Mistral API key before running this script.")
    exit()

client = Mistral(api_key=api_key)

# Pydantic model for Transaction Analyzer agent's response format
class TransactionAnalysisResult(BaseModel):
    summary: str
    flagged_transactions: list
    recommendations: str

print("Creating AI agents for Banking Domain...")

# --- Agent Definitions (Banking Domain) ---

# 1. Customer Onboarding Agent
customer_onboarding_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for onboarding new customers, opening accounts, and verifying identity.",
    name="customer-onboarding-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "open_bank_account",
                "description": "Open a new bank account for a customer with their name, type of account, and initial deposit.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "customer_name": {"type": "string", "description": "Full name of the customer."},
                        "account_type": {"type": "string", "description": "Type of account (e.g., 'checking', 'savings')."},
                        "initial_deposit": {"type": "number", "description": "Initial deposit amount."}
                    },
                    "required": ["customer_name", "account_type", "initial_deposit"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "verify_customer_identity",
                "description": "Verify a customer's identity using their ID document type and number.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "customer_id": {"type": "string", "description": "Unique customer identifier."},
                        "id_type": {"type": "string", "description": "Type of ID document (e.g., 'Passport', 'DriverLicense')."},
                        "id_number": {"type": "string", "description": "ID document number."}
                    },
                    "required": ["customer_id", "id_type", "id_number"]
                }
            }
        }
    ]
)
print(f"Customer Onboarding Agent '{customer_onboarding_agent.name}' created with ID: {customer_onboarding_agent.id}")

# 2. Financial Research Agent
financial_research_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for searching market data, economic reports, and financial news.",
    name="financial-research-agent",
    tools=[
        {"type": "web_search"}, # Still uses web search, but for financial context.
        {
            "type": "function",
            "function": {
                "name": "search_market_data",
                "description": "Search for market data on stocks, bonds, or specific industries.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "query": {"type": "string", "description": "The financial instrument or topic to search for."}
                    },
                    "required": ["query"]
                }
            }
        }
    ]
)
print(f"Financial Research Agent '{financial_research_agent.name}' created with ID: {financial_research_agent.id}")

# 3. Transaction Analyzer Agent
transaction_analyzer_agent = client.beta.agents.create(
    model="mistral-large-latest",
    name="transaction-analyzer-agent",
    description="Agent used to analyze transaction history, identify unusual patterns, and provide insights.",
    instructions="Analyze the provided transaction data, highlight any suspicious or unusual activities, and summarize findings.",
    completion_args={
        "response_format": {
            "type": "json_schema",
            "json_schema": {
                "name": "transaction_analysis_result",
                "schema": TransactionAnalysisResult.model_json_schema(),
            }
        }
    },
    tools=[
        {
            "type": "function",
            "function": {
                "name": "analyze_transactions",
                "description": "Analyze a list of transactions for a given account.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "account_id": {"type": "string", "description": "The account ID to analyze."},
                        "transactions_json": {"type": "string", "description": "A JSON string of transactions, e.g., '[{\"date\": \"2024-06-01\", \"amount\": 1500, \"type\": \"debit\"}]'."}
                    },
                    "required": ["account_id", "transactions_json"]
                }
            }
        }
    ]
)
print(f"Transaction Analyzer Agent '{transaction_analyzer_agent.name}' created with ID: {transaction_analyzer_agent.id}")

# 4. Loan Management Agent
loan_management_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for checking loan eligibility, calculating repayments, and providing loan information.",
    name="loan-management-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "check_loan_eligibility",
                "description": "Check a customer's eligibility for a loan based on income and credit score.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "customer_id": {"type": "string", "description": "Unique customer identifier."},
                        "income": {"type": "number", "description": "Customer's annual income."},
                        "credit_score": {"type": "number", "description": "Customer's credit score."}
                    },
                    "required": ["customer_id", "income", "credit_score"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "calculate_loan_repayment",
                "description": "Calculate estimated monthly repayment for a loan given principal, interest rate, and term.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "principal_amount": {"type": "number", "description": "Loan principal amount."},
                        "interest_rate": {"type": "number", "description": "Annual interest rate (e.g., 0.05 for 5%)."},
                        "loan_term_years": {"type": "number", "description": "Loan term in years."}
                    },
                    "required": ["principal_amount", "interest_rate", "loan_term_years"]
                }
            }
        }
    ]
)
print(f"Loan Management Agent '{loan_management_agent.name}' created with ID: {loan_management_agent.id}")


# 5. Fraud Detection Agent
fraud_detection_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for monitoring transactions and flagging suspicious activities for potential fraud.",
    name="fraud-detection-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "monitor_transactions",
                "description": "Monitor a customer's recent transactions for unusual patterns or high-risk activities.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "account_id": {"type": "string", "description": "Account ID to monitor."},
                        "transaction_details": {"type": "string", "description": "JSON string of recent transactions, e.g., '[{\"amount\": 5000, \"location\": \"Overseas\"}]'."}
                    },
                    "required": ["account_id", "transaction_details"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "flag_suspicious_activity",
                "description": "Flag a specific transaction or activity as suspicious, including reason and severity.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "transaction_id": {"type": "string", "description": "ID of the suspicious transaction."},
                        "reason": {"type": "string", "description": "Reason for flagging (e.g., 'unusual location', 'large amount')."},
                        "severity": {"type": "string", "description": "Severity of suspicion (e.g., 'low', 'medium', 'high')."}
                    },
                    "required": ["transaction_id", "reason", "severity"]
                }
            }
        }
    ]
)
print(f"Fraud Detection Agent '{fraud_detection_agent.name}' created with ID: {fraud_detection_agent.id}")


# 6. Branch Operations Agent
branch_operations_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for managing branch equipment status (ATMs), scheduling maintenance, and reporting issues.",
    name="branch-operations-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "check_atm_status",
                "description": "Check the operational status and last service date for a specific ATM.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "atm_id": {"type": "string", "description": "Unique identifier for the ATM (e.g., 'ATM-001')."}
                    },
                    "required": ["atm_id"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "report_branch_issue",
                "description": "Report an issue within a bank branch (e.g., facility malfunction, equipment problem).",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "branch_id": {"type": "string", "description": "Identifier of the bank branch."},
                        "issue_description": {"type": "string", "description": "A description of the issue."},
                        "severity": {"type": "string", "description": "The severity of the issue (e.g., 'critical', 'minor')."}
                    },
                    "required": ["branch_id", "issue_description", "severity"]
                }
            }
        }
    ]
)
print(f"Branch Operations Agent '{branch_operations_agent.name}' created with ID: {branch_operations_agent.id}")


# 7. Customer Service Staffing Agent
customer_service_staffing_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for managing customer service staff assignments, checking availability, and optimizing schedules.",
    name="customer-service-staffing-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "get_teller_availability",
                "description": "Retrieve the availability of customer service tellers or advisors for a specific date or shift.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "staff_type": {"type": "string", "description": "Type of staff (e.g., 'teller', 'advisor')."},
                        "date": {"type": "string", "description": "Date for availability check in ISO-MM-DD format."}
                    },
                    "required": ["staff_type", "date"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "assign_staff_to_shift",
                "description": "Assign specific staff members to a banking shift or customer interaction.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "staff_ids": {"type": "array", "items": {"type": "string"}, "description": "List of staff member IDs to assign."},
                        "shift_details": {"type": "string", "description": "Description of the shift (e.g., 'Morning Teller Shift')."},
                        "customer_id": {"type": "string", "description": "Optional customer ID if assigning for a specific customer interaction."}
                    },
                    "required": ["staff_ids", "shift_details"]
                }
            }
        }
    ]
)
print(f"Customer Service Staffing Agent '{customer_service_staffing_agent.name}' created with ID: {customer_service_staffing_agent.id}")


# 8. Customer Communication Agent (Refactored from Patient Communication Agent)
customer_communication_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for sending transaction alerts, marketing offers, or general bank updates to customers.",
    name="customer-communication-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "send_transaction_alert",
                "description": "Send a transaction alert to a customer via their preferred contact method.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "customer_id": {"type": "string", "description": "Unique customer identifier."},
                        "transaction_details": {"type": "string", "description": "Details of the transaction (e.g., 'Debit of $50 from checking')."}
                    },
                    "required": ["customer_id", "transaction_details"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "send_marketing_offer",
                "description": "Send a marketing offer or product information to a customer.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "customer_id": {"type": "string", "description": "Unique customer identifier."},
                        "offer_details": {"type": "string", "description": "Content of the marketing offer."}
                    },
                    "required": ["customer_id", "offer_details"]
                }
            }
        }
    ]
)
print(f"Customer Communication Agent '{customer_communication_agent.name}' created with ID: {customer_communication_agent.id}")


# 9. Bank Operations Agent (Refactored from Clinic Operations Agent)
bank_operations_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for overseeing overall bank operations, monitoring customer queues, managing security alerts, and coordinating departments.",
    name="bank-operations-agent",
    tools=[
        {"type": "web_search"}, # For general banking news or external economic data.
        {
            "type": "function",
            "function": {
                "name": "get_queue_status",
                "description": "Retrieve the current customer queue status and wait times in different bank departments.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "department": {"type": "string", "description": "Specific department (e.g., 'teller', 'loans', 'all').", "enum": ["teller", "loans", "customer service", "all"]},
                    },
                    "required": ["department"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "notify_security_alert",
                "description": "Issue a security alert within the bank, specifying the type and location.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "alert_type": {"type": "string", "description": "Type of security alert (e.g., 'suspicious activity', 'system breach')."},
                        "location": {"type": "string", "description": "Location of the alert (e.g., 'Branch 101', 'Online Systems')."}
                    },
                    "required": ["alert_type", "location"]
                }
            }
        }
    ]
)
print(f"Bank Operations Agent '{bank_operations_agent.name}' created with ID: {bank_operations_agent.id}")


print("\nAll new banking-related agents have been defined.")

# --- Mock functions for banking domain tools ---
# These mock functions provide simulated responses for the new banking agents' tools.

def open_bank_account(customer_name: str, account_type: str, initial_deposit: float):
    """MOCK function to open a new bank account."""
    print(f"\n[DEBUG] MOCK CALL: open_bank_account - Name: {customer_name}, Type: {account_type}, Deposit: {initial_deposit}")
    return {"status": "success", "account_number": "ACC-001", "message": f"{account_type} account opened for {customer_name}."}

def verify_customer_identity(customer_id: str, id_type: str, id_number: str):
    """MOCK function to verify customer identity."""
    print(f"\n[DEBUG] MOCK CALL: verify_customer_identity - ID: {customer_id}, Type: {id_type}, Number: {id_number}")
    if customer_id == "CUST-001":
        return {"status": "verified", "message": "Identity verified successfully."}
    return {"status": "failed", "message": "Identity verification failed."}

def search_market_data(query: str):
    """MOCK function to search market data."""
    print(f"\n[DEBUG] MOCK CALL: search_market_data for query: {query}")
    if "AAPL" in query.upper():
        return {"status": "success", "data": {"symbol": "AAPL", "price": 175.50, "change": "+1.20"}, "message": f"Found market data for {query}."}
    return {"status": "success", "data": {}, "message": f"No specific market data found for {query}."}

def analyze_transactions(account_id: str, transactions_json: str):
    """MOCK function to analyze transactions."""
    print(f"\n[DEBUG] MOCK CALL: analyze_transactions for {account_id} with: {transactions_json}")
    try:
        transactions = json.loads(transactions_json)
        summary = f"Analysis for account {account_id}:"
        flagged_transactions = []
        for i, tx in enumerate(transactions):
            if tx.get("amount", 0) > 1000 and tx.get("type") == "debit":
                summary += f" Large debit transaction found on {tx.get('date')} for ${tx.get('amount')}."
                flagged_transactions.append(tx)

        recommendations = "No specific issues detected."
        if flagged_transactions:
            recommendations = "Review flagged transactions for unusual activity."

        return {
            "summary": summary,
            "flagged_transactions": flagged_transactions,
            "recommendations": recommendations
        }
    except json.JSONDecodeError:
        return {"summary": "Invalid JSON format for transactions.", "flagged_transactions": [], "recommendations": ""}

def check_loan_eligibility(customer_id: str, income: float, credit_score: int):
    """MOCK function to check loan eligibility."""
    print(f"\n[DEBUG] MOCK CALL: check_loan_eligibility for {customer_id} - Income: {income}, Score: {credit_score}")
    if income > 50000 and credit_score >= 700:
        return {"status": "eligible", "max_loan_amount": 150000, "message": "Customer is eligible for a loan."}
    return {"status": "not_eligible", "message": "Customer does not meet eligibility criteria."}

def calculate_loan_repayment(principal_amount: float, interest_rate: float, loan_term_years: int):
    """MOCK function to calculate loan repayment."""
    print(f"\n[DEBUG] MOCK CALL: calculate_loan_repayment - Principal: {principal_amount}, Rate: {interest_rate}, Term: {loan_term_years}")
    # Simple interest calculation for mock.
    if interest_rate > 0 and loan_term_years > 0:
        monthly_interest_rate = interest_rate / 12
        num_payments = loan_term_years * 12
        # This is a simplified PMT formula, actual calculation is more complex
        monthly_repayment = principal_amount * monthly_interest_rate / (1 - (1 + monthly_interest_rate)**(-num_payments))
        return {"status": "success", "monthly_repayment": round(monthly_repayment, 2), "message": "Monthly repayment calculated."}
    return {"status": "error", "message": "Invalid loan parameters."}

def monitor_transactions(account_id: str, transaction_details: str):
    """MOCK function to monitor transactions for fraud."""
    print(f"\n[DEBUG] MOCK CALL: monitor_transactions for {account_id} with: {transaction_details}")
    try:
        transactions = json.loads(transaction_details)
        suspicious_activities = []
        for tx in transactions:
            if tx.get("amount", 0) > 5000 and tx.get("location", "").lower() == "overseas":
                suspicious_activities.append(f"Large overseas transaction of ${tx['amount']}.")
        return {"status": "monitoring", "account_id": account_id, "suspicious_activities": suspicious_activities or "None detected."}
    except json.JSONDecodeError:
        return {"status": "error", "message": "Invalid JSON for transaction details."}

def flag_suspicious_activity(transaction_id: str, reason: str, severity: str):
    """MOCK function to flag suspicious activity."""
    print(f"\n[DEBUG] MOCK CALL: flag_suspicious_activity - TX ID: {transaction_id}, Reason: {reason}, Severity: {severity}")
    return {"status": "flagged", "transaction_id": transaction_id, "reason": reason, "severity": severity, "ticket_id": "FRD-9876"}

def check_atm_status(atm_id: str):
    """MOCK function to check ATM status."""
    print(f"\n[DEBUG] MOCK CALL: check_atm_status for {atm_id}")
    if atm_id.upper() == "ATM-001":
        return {"status": "operational", "last_service_date": "2024-06-15", "details": "Cash available, online."}
    return {"status": "offline", "last_service_date": "N/A", "details": "ATM not found or offline."}

def report_branch_issue(branch_id: str, issue_description: str, severity: str):
    """MOCK function to report a branch issue."""
    print(f"\n[DEBUG] MOCK CALL: report_branch_issue for {branch_id}: {issue_description} (Severity: {severity})")
    return {"status": "issue_reported", "ticket_id": "BRN-1122", "branch": branch_id}

def get_teller_availability(staff_type: str, date: str):
    """MOCK function to get teller availability."""
    print(f"\n[DEBUG] MOCK CALL: get_teller_availability for {staff_type} on {date}")
    if staff_type.lower() == "teller" and date == "2024-07-01":
        return {"status": "success", "available_staff_ids": ["TELLER-001", "TELLER-002"]}
    return {"status": "success", "available_staff_ids": []}

def assign_staff_to_shift(staff_ids: list, shift_details: str, customer_id: str = None):
    """MOCK function to assign staff to a shift."""
    print(f"\n[DEBUG] MOCK CALL: assign_staff_to_shift {staff_ids} to '{shift_details}', Customer: {customer_id}")
    return {"status": "success", "message": f"Staff {staff_ids} assigned to '{shift_details}'."}

def send_transaction_alert(customer_id: str, transaction_details: str):
    """MOCK function to send a transaction alert."""
    print(f"\n[DEBUG] MOCK CALL: send_transaction_alert to {customer_id} for: '{transaction_details}'")
    return {"status": "sent", "customer_id": customer_id, "alert_message": transaction_details}

def send_marketing_offer(customer_id: str, offer_details: str):
    """MOCK function to send a marketing offer."""
    print(f"\n[DEBUG] MOCK CALL: send_marketing_offer to {customer_id} with content: '{offer_details}'")
    return {"status": "sent", "customer_id": customer_id, "offer": offer_details}

def get_queue_status(department: str):
    """MOCK function to get customer queue status."""
    print(f"\n[DEBUG] MOCK CALL: get_queue_status for department: {department}")
    if department.lower() == "teller":
        return {"status": "success", "department": "Teller", "customer_count": 8, "avg_wait_time_minutes": 10}
    elif department.lower() == "loans":
        return {"status": "success", "department": "Loans", "customer_count": 3, "avg_wait_time_minutes": 25}
    return {"status": "success", "department": department, "customer_count": 0, "avg_wait_time_minutes": 0}

def notify_security_alert(alert_type: str, location: str):
    """MOCK function to issue a security alert."""
    print(f"\n[DEBUG] MOCK CALL: notify_security_alert - Type: {alert_type}, Location: {location}")
    return {"status": "notification_issued", "alert_type": alert_type, "location": location, "timestamp": time.time()}


# --- Master Tool Executor Mapping (Updated for Banking Domain) ---
tool_executor = {
    "open_bank_account": open_bank_account,
    "verify_customer_identity": verify_customer_identity,
    "search_market_data": search_market_data,
    "analyze_transactions": analyze_transactions,
    "check_loan_eligibility": check_loan_eligibility,
    "calculate_loan_repayment": calculate_loan_repayment,
    "monitor_transactions": monitor_transactions,
    "flag_suspicious_activity": flag_suspicious_activity,
    "check_atm_status": check_atm_status,
    "report_branch_issue": report_branch_issue,
    "get_teller_availability": get_teller_availability,
    "assign_staff_to_shift": assign_staff_to_shift,
    "send_transaction_alert": send_transaction_alert,
    "send_marketing_offer": send_marketing_offer,
    "get_queue_status": get_queue_status,
    "notify_security_alert": notify_security_alert,
    "internal_web_search_tool": lambda *args, **kwargs: "Mock web search: Financial information retrieved."
}

# --- Function to standardize tools for client.chat.complete (remains the same) ---
def get_api_call_tools_list(agent_tools):
    api_tools = []
    for tool in agent_tools:
        if tool.type == 'function':
            api_tools.append(tool.model_dump())
        elif tool.type == 'web_search':
            api_tools.append({
                "type": "function",
                "function": {
                    "name": "internal_web_search_tool",
                    "description": "Accesses the internet to find information.",
                    "parameters": {
                        "type": "object",
                        "properties": {}
                    }
                }
            })
    return api_tools

# --- Test Case Execution for Banking Agents ---

print("\n--- Executing Test Cases for Banking Agents (via chat completions) ---")

test_cases = [
    {
        "agent": customer_onboarding_agent,
        "name": "Customer Onboarding Agent",
        "query": "Open a new checking account for John Smith with an initial deposit of $1000.",
        "expected_tool_call": "open_bank_account"
    },
    {
        "agent": customer_onboarding_agent,
        "name": "Customer Onboarding Agent",
        "query": "Verify identity for customer CUST-001, using Passport number P1234567.",
        "expected_tool_call": "verify_customer_identity"
    },
    {
        "agent": financial_research_agent,
        "name": "Financial Research Agent",
        "query": "Search for current stock data for Apple (AAPL).",
        "expected_tool_call": "search_market_data"
    },
    {
        "agent": transaction_analyzer_agent,
        "name": "Transaction Analyzer Agent",
        "query": "Analyze transactions for account ACC-001: [{\"date\": \"2024-06-01\", \"amount\": 1500, \"type\": \"debit\", \"description\": \"Online Purchase\"}, {\"date\": \"2024-06-05\", \"amount\": 2500, \"type\": \"credit\", \"description\": \"Salary\"}, {\"date\": \"2024-06-10\", \"amount\": 1200, \"type\": \"debit\", \"description\": \"Utility Bill\"}]",
        "expected_tool_call": "analyze_transactions"
    },
    {
        "agent": loan_management_agent,
        "name": "Loan Management Agent",
        "query": "Check loan eligibility for customer CUST-001 with income $60000 and credit score 720.",
        "expected_tool_call": "check_loan_eligibility"
    },
    {
        "agent": loan_management_agent,
        "name": "Loan Management Agent",
        "query": "Calculate monthly repayment for a $100000 loan at 4.5% interest over 30 years.",
        "expected_tool_call": "calculate_loan_repayment"
    },
    {
        "agent": fraud_detection_agent,
        "name": "Fraud Detection Agent",
        "query": "Monitor transactions for account ACC-001: [{\"amount\": 6000, \"location\": \"Overseas\", \"id\": \"TXN-005\"}, {\"amount\": 100, \"location\": \"Local\", \"id\": \"TXN-006\"}]",
        "expected_tool_call": "monitor_transactions"
    },
    {
        "agent": fraud_detection_agent,
        "name": "Fraud Detection Agent",
        "query": "Flag transaction TXN-005 as suspicious due to large overseas amount (High severity).",
        "expected_tool_call": "flag_suspicious_activity"
    },
    {
        "agent": branch_operations_agent,
        "name": "Branch Operations Agent",
        "query": "Check status of ATM-001.",
        "expected_tool_call": "check_atm_status"
    },
    {
        "agent": branch_operations_agent,
        "name": "Branch Operations Agent",
        "query": "Report critical issue at Branch 101: Power outage affecting systems.",
        "expected_tool_call": "report_branch_issue"
    },
    {
        "agent": customer_service_staffing_agent,
        "name": "Customer Service Staffing Agent",
        "query": "Are there any tellers available on July 1st, 2024?",
        "expected_tool_call": "get_teller_availability"
    },
    {
        "agent": customer_service_staffing_agent,
        "name": "Customer Service Staffing Agent",
        "query": "Assign TELLER-001 and TELLER-002 to Morning Teller Shift.",
        "expected_tool_call": "assign_staff_to_shift"
    },
    {
        "agent": customer_communication_agent,
        "name": "Customer Communication Agent",
        "query": "Send transaction alert to CUST-001: 'Debit of $50 from checking on 2024-06-21'.",
        "expected_tool_call": "send_transaction_alert"
    },
    {
        "agent": customer_communication_agent,
        "name": "Customer Communication Agent",
        "query": "Send marketing offer to CUST-001: 'Special low interest rate on new credit cards.'",
        "expected_tool_call": "send_marketing_offer"
    },
    {
        "agent": bank_operations_agent,
        "name": "Bank Operations Agent",
        "query": "What's the current customer queue status in the teller department?",
        "expected_tool_call": "get_queue_status"
    },
    {
        "agent": bank_operations_agent,
        "name": "Bank Operations Agent",
        "query": "Notify security alert: 'Suspicious activity detected in online systems'.",
        "expected_tool_call": "notify_security_alert"
    },
    {
        "agent": financial_research_agent, # Reusing for web search example
        "name": "Financial Research Agent (Web Search Example)",
        "query": "Latest news on global inflation trends.",
        "expected_tool_call": "internal_web_search_tool"
    }
]

for test_case in test_cases:
    agent_to_test = test_case["agent"]
    agent_name = test_case["name"]
    user_query = test_case["query"]
    expected_tool_call_name = test_case["expected_tool_call"] # This is for tracking, not direct control.

    print(f"\n--- Executing Test Case for the {agent_name} ---")
    print(f"User: {user_query}")

    conversation_history = []
    conversation_history.append({"role": "user", "content": user_query})

    try:
        # Get the API-compatible tool list for the current agent.
        api_call_tools_list = get_api_call_tools_list(agent_to_test.tools)

        print(f" [DEBUG] Sending initial user query to the {agent_name}...")
        response_turn1 = client.chat.complete(
            model=agent_to_test.model,
            messages=conversation_history,
            tools=api_call_tools_list,
        )

        assistant_message_turn1 = response_turn1.choices[0].message
        conversation_history.append(assistant_message_turn1.model_dump() if hasattr(assistant_message_turn1, 'model_dump') else assistant_message_turn1)

        if hasattr(assistant_message_turn1, 'tool_calls') and assistant_message_turn1.tool_calls:
            print(f"\n{agent_name} proposed tool calls (Turn 1):")
            for tool_call in assistant_message_turn1.tool_calls:
                print(f" Tool Name: {tool_call.function.name}")
                print(f" Tool Arguments (JSON string): {tool_call.function.arguments}")

                tool_output_content = None
                # Check if the proposed tool exists in our local executor mapping.
                if tool_call.function.name in tool_executor:
                    try:
                        args = json.loads(tool_call.function.arguments)
                        # Execute the local mock function based on its name.
                        tool_output = tool_executor[tool_call.function.name](**args)
                        tool_output_content = json.dumps(tool_output)
                        print(f" [DEBUG] Local MOCK {tool_call.function.name} executed. Output: {tool_output_content}")
                    except json.JSONDecodeError as e:
                        print(f" [ERROR] Failed to parse tool arguments for {tool_call.function.name}: {e}")
                        tool_output_content = json.dumps({"error": f"Failed to parse arguments: {e}"})
                    except Exception as e:
                        print(f" [ERROR] Error executing local mock {tool_call.function.name}: {e}")
                        tool_output_content = json.dumps({"error": f"Tool execution failed: {e}"})
                else:
                    print(f" [DEBUG] Unhandled tool call: {tool_call.function.name}")
                    tool_output_content = json.dumps({"error": "Tool not handled by client-side executor."})

                # Add the tool output message to the conversation history.
                # This is crucial for the model to "see" the result of the tool call.
                conversation_history.append(
                    {
                        "role": "tool",
                        "name": tool_call.function.name,
                        "content": tool_output_content,
                        "tool_call_id": tool_call.id # Links the tool output to the specific call request.
                    }
                )
                print(f" [DEBUG] Tool output for '{tool_call.function.name}' added to history.")

            # Second turn: Send the conversation history (including tool outputs) back to the model.
            # The model will then generate a final response based on the tool's output.
            print(f"\n [DEBUG] Sending conversation history with tool outputs back for final response from {agent_name}...")
            final_response = client.chat.complete(
                model=agent_to_test.model,
                messages=conversation_history,
                tools=api_call_tools_list, # Tools must be provided in all calls if they are part of the conversation context.
            )

            final_assistant_message = final_response.choices[0].message
            print(f"\n{agent_name}'s Final Response:")
            print(final_assistant_message.content)
            # Add the final assistant response to history (optional for a single-turn demo, but good practice).
            conversation_history.append(final_assistant_message.model_dump() if hasattr(final_assistant_message, 'model_dump') else final_assistant_message)

        else:
            # If no tool calls were proposed in the first turn, print the direct response.
            print(f"\n{agent_name}'s initial response (no tool calls proposed):")
            print(assistant_message_turn1.content)

    except Exception as e:
        print(f"\nAn error occurred during {agent_name} interaction: {e}")
        print("Please check your API key, model availability, network connection, or SDK version.")
        print("If you continue to experience errors, a complete restart of your Python environment (e.g., Colab runtime) might help.")

print("\n--- All test cases execution complete. ---")


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Creating AI agents for Banking Domain...
Customer Onboarding Agent 'customer-onboarding-agent' created with ID: ag_06856b43ae5370358000b4229f698cef
Financial Research Agent 'financial-research-agent' created with ID: ag_06856b43b37b745880005aed7c221891
Transaction Analyzer Agent 'transaction-analyzer-agent' created with ID: ag_06856b43b6ae77488000fea5bc577b90
Loan Management Agent 'loan-management-agent' created with ID: ag_06856b43bbdd750f8000469612c6ba4b
Fraud Detection Agent 'fraud-detection-agent' created with ID: ag_06856b43c5a070748000968f2e807686
Branch Operations Agent 'branch-operations-agent' created with ID: ag_06856b43c9cf701680000451065c7214
Customer Service Staffing Agent 'customer-service-staffing-agent' created with ID: ag_06856b43ccff7c76800010699ef953ce
Customer Communication Agent 'customer-communication-agent' created with ID: ag_06856b4